# 4 Methodology:
Our methodology is split into a section for data preprocessing and then sections for each of our research questions.

## 4.1 Data Preprocessing
Please note: All preprocessed data will be included in the zipped file zubmitted to canvas. Links to the original public datasets will be included in the file original_data.txt in that directory. 
### 4.1.1 The Yelp Dataset
The Yelp dataset is a set of JSON files containing a data architecture similar to nested dictionaries in python. Our analysis only required the business.json (138 mb in size) file, so we only preprocessed that file. A function was written using the json python module to read in the business.json file objects into nested dictionaries in python, filtering the inner dictionaries of attributes of interest. We saved this preprocessed subset of the data as a .csv for further analysis.
### 4.1.2 The Zillow Dataset
A python script was written for preprocessing the Zillow dataset. This dataset contains a rich depth of housing information on a per zip code basis. This is read into a pandas data frame and filtered such that the data frame contains only columns for city, zhvi (Zillow Home Value Index - a metric on current home value), and zip code. This data is then further filtered to the target city, which for the sake of this project is Las Vegas. This file is read into another script where the statistical analysis is performed. Furthermore, this script contains a function that is used to create a list of zip codes from the zillow data, which was used to filter the IRS dataset (see below). 
### 4.1.3 The IRS Dataset
The IRS data used, known as IRS SOI Tax Data, provides data, on a zip code
level, for taxpayers in a given state, in this case Nevada. For further analysis we were interested in exploiting an mean income per zip code from this dataset. The raw data provides total income over 5 tax bins for each zip code, so we summed up these totals and divided by the total number of taxpayers in each zip code. Filtering based on the zip codes from the zillow dataset, the resultant mean income data was stored as a .csv and used for further analysis. 
	4.1.4 Geographical GeoJSON of Nevada Zip Code Boundaries Dataset
This dataset was read in using the geopandas module and filtered to add boundaries for the analyzed Las Vegas zip codes to our map visualization figures. No preprocessing was performed on this data.


## 4.2 Are there any trends across zip code areas concerning restaurant pricing, household income, and housing cost?
First, the csv files containing the preprocessed data are read into pandas data frames. These pandas data frames are then sorted by zip code in ascending order. Then, a dictionary is created with zip codes as keys and average price of restaurants as the respective values. This dictionary is then converted to a data frame and merged with the data frames containing the zhvis per zip code and the mean income per zip code. This newly generated data frame contains zhvi, mean income, and average price point per zip code. Then, each zip code is used as a datapoint to generate a correlation matrix using the pearson’s correlation coefficient. Scatterplots are then generated between each of the variables using the matplotlib library. Next, the Las Vegas strip zip code 89109 is removed from the dataset, as it is an outlier in all of our datasets and a correlation matrix using pearson’s correlation coefficient is generated again.

## 4.3 Can we predict average restaurant pricing in a zip code area from median house value and average household income?
First, the zip code containing the Las Vegas Strip, an outlier, is removed from the data frame created above containing zhvi, mean income, and average restaurant price per zip code. This was done, as the Las Vegas Stip has a substanially larger proportion of fancy restaurants and a subsequently large number ratings. Since the point of the linear regression model is to predict average price point based on income and median house value (zhvi), we decided that removing the outlier would generate a model that is more applicable to a larger proportion of the zipcodes in the city of Las Vegas.

The sklearn module is used to generate two linear regression models. The first fit a model to predict average restaurant price from income per zip code and the second to predict average restaurant price from average home price (zillow home value index) per zip code. We evaluated the performance of prediction of the two models using the R-squared metric from regression. Although sklearn has a function to compute this metric, the formula it uses is specific to more complex regression than we performed. Therefore, we leveraged the fact that for linear regression the R-squared metric is simply the square of the pearson’s correlation coefficient, so we computed the metric this way. We also plotted, using matplotlib, the original data versus the computed regression line to visually inspect the goodness of fit.

## 4.4 Does restaurant star rating across price point level vary for different zip code areas? What are trends in such variance across zip code areas and how does it relate to socioeconomic factors such as median house value and average household income?

### 4.4.1 Correlation 
First, the data frame generated for research question 1 is used to create a new data frame that contains the average rating (yelp stars out of 5) per price point per zip code. This data frame has columns for zip code, price point 1, price point 2, price point 3, price point 4, zhvi, and mean income. 

A correlation matrix is then generated using Pearson’s correlation coefficients for the data frame with the average rating per price point per zip code. Scatter plots of mean income versus average rating are then generated using the matplotlib library for each of the four price points.

Another data frame is generated that contains the proportion of restaurants per price point per zip code:
$$ proportion = \frac{Number \ Restaurants \ in \ Price \ Point}{Total \ Number \ Restaurants \ in \ Zip \ Code} $$ This data frame has columns for zip code, price point 1, price point 2, price point 3, price point 4, zhvi, and mean income.

A correlation matrix is then generated using Pearson’s correlation coefficients for the data frame with the proportion of restaurants per price point per zip code. Scatter plots of mean income versus restaurant proportion are then generated using the matplotlib library for each of the four price points. Then, comparative boxplots are generated for average rating and restaurant proportion clustered by price point.

This analysis is then completed two more times after (1) the removal of zip codes found to be dependent in the chi-square analysis and after (2) the removal of zip codes found to be dependent in the chi-square analysis and the Las Vegas Strip outlier.

### 4.4.2 Chi-Square Analysis 
To compute chi-square tests of independence, the Scipy Stats module was used. A chi-square test of independence is computed using the formula for chi-square, $$\chi^2=\sigma \frac{(obs_i - exp_i)^2}{exp_i}$$ which compares the observed counts of a variable across different bins (values of a second variable) to the expected values of the variable if the variable is independent of the bin. The p-value is then calculated from the chi-square statistic and the degrees of freedom (which is based on the number of bins). A higher chi-square value indicates greater deviation of the observed counts from the expected values; this corresponds to a smaller p-value, which indicates a smaller probability that such deviations occurred by chance. For our chi-square tests, we used an alpha level of 0.01, which gave the chi-square results 99% confidence level. 

Chi-square tests were performed on the summary (total) of all zip code price point levels using the Scipy Stats “chisquare” function, using observed and expected data computed from the summary data. 

Chi-square tests were performed for each zip code using the created “chiDict” function and by iterating through the list of zip codes. The “chiDict” function was used to create a dictionary including the zip code,  chi-square statistic value, and p-value as well as a boolean value of whether or not the p-value is below alpha (which we set to 0.01). This function again relied on the Scipy Stats “chisquare” function to determine the chi-square statistic and p-value. For each zip code, the “chiDict” function was used to create a dictionary for the zip code, which was appended to a list of dictionaries that was later converted to a data frame.

## 4.5 Geopandas map visualization
In order to support analysis of the variation of various metrics across Las Vegas zip codes we wanted to visualize the spatial distributions of such information across the city. This was completed by using the geopandas python module to create what are known as choropleth maps. These maps depict the value of a certain metric as a relative color intensity of the map shape, in this case the zip code boundary. Creation of these visualizations required creating dictionaries mapping from zip code to the metric of interest and the following the geopandas procedure to plot the map. All maps are depicted alongside an intensity spectrum where smaller values are show as lighter toned colors. Unfortunately, we could not navigate the mayhem of layered matplotlib objects to add proper labels to the colorbars.
